In [1]:
%%capture
%cd ../
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import logging

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score
from tqdm import tqdm
from xgboost import XGBClassifier
import pandas as pd
pd.set_option('display.max_rows', 150)

from src.constants import symp_cols
from src.label import convert_to_binary_symptom_labels, get_symptom_labels, get_label_distribution
from src.prepare.filter import (
    drop_highly_missing_features, 
    drop_samples_outside_study_date, 
    drop_samples_with_no_targets,
    drop_unused_drug_features,
    keep_only_one_per_week,
)
from src.prepare.engineer import collapse_rare_categories, get_change_since_prev_session, get_missingness_features
from src.prepare.pipeline import PrepSympData
from src.prepare.prep import fill_missing_data
from src.prepare.split import create_temporal_cohort
from src.summarize import feature_summary, pre_and_post_treatment_missingness_summary
from src.util import get_nunique_categories, get_nmissing, initialize_folders

from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

initialize_folders()

logging.basicConfig(
    filename=f"./logs/{datetime.now().strftime('%Y-%m-%d %H.%M.%S')}_symptom_target.log",
    level=logging.INFO, 
    format='%(asctime)s %(levelname)s:%(message)s', 
    datefmt='%Y-%m-%d %H:%M:%S'
)

In [3]:
# Load data
df = pd.read_parquet('data/treatment_centered_clinical_dataset.parquet.gzip')

In [4]:
# scoring increase thresholds for determining symptom deterioration
target_pt_increases = [1, 3]

# Prep Data

In [5]:
# filter out dates before 2014 and after 2020
df = drop_samples_outside_study_date(df)

# keep only the first treatment session of a given week
df = keep_only_one_per_week(df)

# get the change in measurement since previous assessment
df = get_change_since_prev_session(df)

# extract labels
symp = pd.read_parquet('./data/external/symptom.parquet.gzip')
df = get_symptom_labels(df, symp)
for pt_increase in target_pt_increases:
    scoring_map = {symp: pt_increase for symp in symp_cols if symp != 'patient_ecog'}
    df = convert_to_binary_symptom_labels(df, scoring_map=scoring_map)

Getting symptom labels...: 100%|██████████| 1097/1097 [00:05<00:00, 184.88it/s]


In [6]:
pretreatment_cols = [f'esas_{symp}' for symp in ['nausea', 'appetite', 'pain', 'shortness_of_breath', 'tiredness', 'drowsiness', 'depression', 'anxiety', 'well_being']]
posttreatment_cols = [f'target_{col}_survey_date' for col in pretreatment_cols]
event_cols = [f'target_{col}_3pt_change' for col in pretreatment_cols]
dev_cohort, test_cohort = create_temporal_cohort(df, split_date='2017-10-01')
for name, cohort in {'test': test_cohort, 'dev': dev_cohort}.items():
    result = pre_and_post_treatment_missingness_summary(cohort, pretreatment_cols, posttreatment_cols, event_cols)
    result.to_csv(f'./result/tables/missingness_summary_{name}_cohort.csv')
result[['any_missingness_trt', 'target_missingness_trt', 'event_rate_trt', 'any_missingness_mrn', 'event_rate_mrn']]

,any_missingness_trt,target_missingness_trt,event_rate_trt,any_missingness_mrn,event_rate_mrn
esas_nausea,10392 (41.6),4647 (24.1),1456 (10.0),2061 (70.7),758 (30.1)
esas_appetite,10367 (41.5),4621 (24.0),1988 (13.6),2054 (70.4),972 (38.6)
esas_pain,10356 (41.4),4633 (24.0),1606 (11.0),2061 (70.7),840 (33.3)
esas_shortness_of_breath,10366 (41.5),4627 (24.0),1165 (8.0),2063 (70.7),630 (25.1)
esas_tiredness,10367 (41.5),4632 (24.0),2009 (13.7),2062 (70.7),1017 (40.4)
esas_drowsiness,10401 (41.6),4652 (24.2),1963 (13.4),2062 (70.7),990 (39.3)
esas_depression,10400 (41.6),4637 (24.1),1120 (7.7),2064 (70.8),624 (24.8)
esas_anxiety,10403 (41.6),4643 (24.1),1147 (7.9),2068 (70.9),636 (25.3)
esas_well_being,10551 (42.2),4666 (24.4),1682 (11.6),2078 (71.3),884 (35.2)
Mean,10400 (41.6),4639 (24.1),1570 (10.8),2063 (70.7),816 (32.4)


In [7]:
# filter out sessions without any labels
target_cols = 'target_' + pd.Index(symp_cols) + '_change'
df = drop_samples_with_no_targets(df, target_cols)

# drop drug features that were never used
df = drop_unused_drug_features(df)

In [8]:
get_nunique_categories(df)

,regimen,intent
Number of Unique Categories,107,4


In [9]:
nmissing = get_nmissing(df)
nmissing[~nmissing.index.str.endswith('_date')].tail(20)

,Missing (N),Missing (%)
aspartate_aminotransferase_change,14043,61.562
alanine_aminotransferase_change,14224,62.356
alkaline_phosphatase_change,14261,62.518
total_bilirubin_change,14311,62.737
glucose_change,14450,63.347
lactate_dehydrogenase_change,14752,64.671
phosphate_change,14927,65.438
albumin_change,15220,66.722
eosinophil_change,16140,70.755
bicarbonate,17600,77.156


In [10]:
# fill missing data that can be filled heuristically
df = fill_missing_data(df)

# drop features with high missingness
keep_cols = df.columns[df.columns.str.contains('target_')]
df = drop_highly_missing_features(df, missing_thresh=75, keep_cols=keep_cols)

# create missingness features
df = get_missingness_features(df)

# collapse rare morphology and cancer sites into 'Other' category
df = collapse_rare_categories(df, catcols=['cancer_site', 'morphology'])

In [11]:
prep = PrepSympData()
X, Y, metainfo = prep.run_pipeline(df, split_date='2017-09-30', target_pt_increases=target_pt_increases)
# clean up Y
Y = Y[[col for col in Y.columns if col.endswith('pt_change')]]
for substr in ['target_', 'esas_']: Y.columns = Y.columns.str.replace(substr, '')

In [12]:
train_mask, valid_mask, test_mask = metainfo['split'] == 'Train', metainfo['split'] == 'Valid', metainfo['split'] == 'Test'
X_train, X_valid, X_test = X[train_mask], X[valid_mask], X[test_mask]
Y_train, Y_valid, Y_test = Y[train_mask], Y[valid_mask], Y[test_mask]

# Describe Data

In [13]:
count = pd.DataFrame({
    'Number of sessions': metainfo.groupby('split').apply(len, include_groups=False), 
    'Number of patients': metainfo.groupby('split')['mrn'].nunique()}
).T
count['Total'] = count.sum(axis=1)
print(f'\n{count.to_string()}')


split               Test  Train  Valid  Total
Number of sessions  5570  11723   2974  20267
Number of patients   960   1815    454   3229


In [14]:
get_label_distribution(Y, metainfo, with_respect_to='sessions').sort_index()

Test             Train            Valid        \
                                   0     1   -1      0     1   -1     0     1   
anxiety_1pt_change              3854  1662   54   8137  3392  194  2089   832   
anxiety_3pt_change              4947   417  206  10281   919  523  2628   221   
appetite_1pt_change             3663  1817   90   7464  4066  193  1938   988   
appetite_3pt_change             4525   752  293   9531  1580  612  2431   389   
depression_1pt_change           4009  1519   42   8283  3270  170  2100   839   
depression_3pt_change           5032   391  147  10438   889  396  2673   224   
drowsiness_1pt_change           3438  2099   33   7198  4374  151  1855  1068   
drowsiness_3pt_change           4599   756  215   9714  1559  450  2470   389   
nausea_1pt_change               4031  1496   43   8202  3367  154  2113   839   
nausea_3pt_change               4974   502   94  10305  1161  257  2654   276   
pain_1pt_change                 3861  1686   23   7897  3694  132  2058   883   
pain_3pt_change                 4834   570  166  10026  1296  401  2597   295   
shortness_of_breath_1pt_change  3976  1548   46   8148  3435  140  2106   845   
shortness_of_breath_3pt_change  4918   419  233  10418   938  367  2684   220   
tiredness_1pt_change            3155  2376   39   6614  4927  182  1708  1228   
tiredness_3pt_change            4417   746  407   9268  1617  838  2407   376   
well_being_1pt_change           3337  2190   43   7021  4397  305  1868  1032   
well_being_3pt_change           4668   644  258   9695  1355  673  2512   309   

                                     Total              
                                 -1      0     1    -1  
anxiety_1pt_change               53  14080  5886   301  
anxiety_3pt_change              125  17856  1557   854  
appetite_1pt_change              48  13065  6871   331  
appetite_3pt_change             154  16487  2721  1059  
depression_1pt_change            35  14392  5628   247  
depression_3pt_change            77  18143  1504   620  
drowsiness_1pt_change            51  12491  7541   235  
drowsiness_3pt_change           115  16783  2704   780  
nausea_1pt_change                22  14346  5702   219  
nausea_3pt_change                44  17933  1939   395  
pain_1pt_change                  33  13816  6263   188  
pain_3pt_change                  82  17457  2161   649  
shortness_of_breath_1pt_change   23  14230  5828   209  
shortness_of_breath_3pt_change   70  18020  1577   670  
tiredness_1pt_change             38  11477  8531   259  
tiredness_3pt_change            191  16092  2739  1436  
well_being_1pt_change            74  12226  7619   422  
well_being_3pt_change           153  16875  2308  1084

In [15]:
get_label_distribution(Y, metainfo, with_respect_to='patients').sort_index()

Test      Train       Valid      Total      
                                  1    0     1     0     1    0     1     0
anxiety_1pt_change              611  349  1187   628   300  154  2098  1131
anxiety_3pt_change              234  726   510  1305   125  329   869  2360
appetite_1pt_change             636  324  1333   482   328  126  2297   932
appetite_3pt_change             386  574   777  1038   187  267  1350  1879
depression_1pt_change           569  391  1160   655   291  163  2020  1209
depression_3pt_change           226  734   497  1318   126  328   849  2380
drowsiness_1pt_change           723  237  1408   407   349  105  2480   749
drowsiness_3pt_change           403  557   785  1030   200  254  1388  1841
nausea_1pt_change               554  406  1173   642   283  171  2010  1219
nausea_3pt_change               276  684   609  1206   141  313  1026  2203
pain_1pt_change                 643  317  1290   525   301  153  2234   995
pain_3pt_change                 317  643   668  1147   167  287  1152  2077
shortness_of_breath_1pt_change  554  406  1165   650   283  171  2002  1227
shortness_of_breath_3pt_change  241  719   507  1308   119  335   867  2362
tiredness_1pt_change            777  183  1488   327   377   77  2642   587
tiredness_3pt_change            404  556   820   995   194  260  1418  1811
well_being_1pt_change           736  224  1420   395   354  100  2510   719
well_being_3pt_change           351  609   707  1108   171  283  1229  2000

In [16]:
# Feature Characteristics
x = prep.ohe.encode(df.loc[X_train.index].copy(), verbose=False) # get original (non-normalized, non-imputed) data one-hot encoded
x = x[[col for col in x.columns if not (col in metainfo.columns or col.startswith('target'))]]
feature_summary(x, save_path='result/tables/feature_summary.csv').sample(10, random_state=42)

,Features,Group,Mean (SD),Missingness (%)
203,Regimen LU-NIVOLUMAB (SAP),Treatment,0.004 (0.063),0.0
211,Regimen TRIAL,Treatment,0.084 (0.277),0.0
214,Intent of Systemic Treatment Neoadjuvant,Treatment,0.057 (0.231),0.0
175,Regimen GI-PANITUMUMAB,Treatment,0.011 (0.104),0.0
48,"Morphology ICD-0-3 848, Cystic, mucinous, and ...",Cancer,0.041 (0.198),0.0
92,White Blood Cell (x10e9/L),Laboratory,6.004 (3.412),30.5
180,Regimen HN-CETUXIMAB (LOAD),Treatment,0.001 (0.029),0.0
73,Glucose (mmol/L),Laboratory,6.970 (2.593),45.6
204,Regimen LU-PACLICARBO,Treatment,0.004 (0.060),0.0
215,Intent of Systemic Treatment Palliative,Treatment,0.732 (0.443),0.0


# Train Model

In [17]:
# Logistic Regression
targets = Y.columns
LR_params = {'C': 0.3, 'penalty': 'l2', 'class_weight': 'balanced', 'max_iter': 2000, 'random_state': 42} # 'solver': 'saga', 
LR_model = {target: LogisticRegression(**LR_params) for target in targets}
XGB_params = dict(n_estimators=100, max_depth=6, learning_rate=0.01, min_child_weight=6, random_state=42)
XGB_model = {target: XGBClassifier(**XGB_params) for target in targets}
for target in tqdm(targets):
    mask = Y_train[target] != -1
    LR_model[target].fit(X_train[mask], Y_train.loc[mask, target])
    XGB_model[target].fit(X_train[mask], Y_train.loc[mask, target])

  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [02:01<00:00,  6.76s/it]


In [18]:
def evaluate(model, X, Y):
    result = {}
    for target, label in Y.items():
        mask = label != -1
        # check model.classes_ to confirm prediction of positive label is at index 1
        pred = model[target].predict_proba(X[mask])[: ,1]
        auprc = average_precision_score(label[mask], pred)
        auroc = roc_auc_score(label[mask], pred)
        result[target] = {'AUPRC': auprc, 'AUROC': auroc}
    return pd.DataFrame(result).T

In [20]:
evaluate(LR_model, X_valid, Y_valid)

,AUPRC,AUROC
pain_1pt_change,0.475862,0.655851
tiredness_1pt_change,0.627286,0.678208
nausea_1pt_change,0.453932,0.671093
depression_1pt_change,0.401786,0.651748
anxiety_1pt_change,0.388799,0.617675
drowsiness_1pt_change,0.561692,0.685365
appetite_1pt_change,0.563517,0.700907
well_being_1pt_change,0.563339,0.680049
shortness_of_breath_1pt_change,0.408388,0.653472
pain_3pt_change,0.292664,0.730638


In [21]:
evaluate(LR_model, X_test, Y_test)

,AUPRC,AUROC
pain_1pt_change,0.464188,0.637326
tiredness_1pt_change,0.611927,0.673078
nausea_1pt_change,0.443644,0.671883
depression_1pt_change,0.384616,0.626949
anxiety_1pt_change,0.401322,0.610335
drowsiness_1pt_change,0.564664,0.675702
appetite_1pt_change,0.517947,0.663640
well_being_1pt_change,0.570521,0.665429
shortness_of_breath_1pt_change,0.368694,0.600363
pain_3pt_change,0.226392,0.675176


In [22]:
evaluate(XGB_model, X_valid, Y_valid)

,AUPRC,AUROC
pain_1pt_change,0.497675,0.683565
tiredness_1pt_change,0.620820,0.689137
nausea_1pt_change,0.498506,0.706147
depression_1pt_change,0.475036,0.707769
anxiety_1pt_change,0.434293,0.676690
drowsiness_1pt_change,0.558526,0.703425
appetite_1pt_change,0.584689,0.728845
well_being_1pt_change,0.550627,0.692945
shortness_of_breath_1pt_change,0.410443,0.650389
pain_3pt_change,0.299741,0.751617


In [19]:
evaluate(XGB_model, X_test, Y_test)

,AUPRC,AUROC
pain_1pt_change,0.479161,0.662992
tiredness_1pt_change,0.604046,0.666222
nausea_1pt_change,0.445009,0.675159
depression_1pt_change,0.399748,0.652901
anxiety_1pt_change,0.430365,0.644815
drowsiness_1pt_change,0.569400,0.683269
appetite_1pt_change,0.516838,0.676955
well_being_1pt_change,0.554816,0.666361
shortness_of_breath_1pt_change,0.405333,0.637055
pain_3pt_change,0.244697,0.700301
